# Flow Counter

* This notebook is used to count flows, in respect to each geochem group, from different wells.
-----------------------------------------------------------------------------------------------------------------   
* Main parts include:
     * A flow counter algorithm
     * A function generalizing the flow-counter algorithm -- TODO

## Importing libraries and data

In [1]:
# Importing libraries
import pandas as pd

In [2]:
# Reading .csv
data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';')
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"11,2","31,36","0,589285714"
1,1-AV-001-PR,Paranapanema,Simple lava (basic | massive interior),"18,48",NaN,NaN
2,1-AV-001-PR,Paranapanema,Simple lava (basic | L. crust),"1,68",NaN,NaN
3,1-AV-001-PR,Paranapanema,Siliciclastics,"1,23","1,23",NaN
4,1-AV-001-PR,Paranapanema,Simple lava (basic | U. crust),"29,81","88,48","0,643196203"


## Flow Counter Algorithm
* This code is used to run throught the WellCad data and counting the # flows per geochemical group.
-----------------------------------------------------------------------------------------------------------------   
* Things considered to indicate flow "change":
    * Upper crust of lava flow
    * Compound flows are counted as singular flow
    * Interbeds indicate flow changes
    * Unit changes indicate flow changes
-----------------------------------------------------------------------------------------------------------------    
* Output:
    * A flow_list list which contains the exact same number of values as the input Pandas DataFrame. 
    * This list can then be appended to the given dataframe to generate a flow# column.

In [6]:
# Flow Counter v2 -- Accounting for Unit Intercalation


# Setting up variables
flow_counter = 0
flow_list = []
flow_dict = {}

# Algorithm
for i in range(len(data)):
    if i==0: # Setting up the first iteration
        flow_counter += 1
        flow_dict[data['Facies'].iloc[i]] = flow_counter
        flow_list.append(flow_counter)
    elif i == (len(data)-1): # Checking for the end of the list (working around out-of-bounds bug)
        flow_list.append(flow_counter)
        break
    elif data['Facies'].iloc[i] == "Siliciclastics" or data['Facies'].iloc[i] == "Volcaniclastics (undiff.)": # Check for siliciclastic OR volcaniclastic; append a "-" for these units:
        flow_list.append('-')        
    else: # If we're indeed dealing with lava flow data  
        if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before
            if data['Facies'].iloc[i] == 'Simple lava (basic | L. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)': # Check for Simple lower crust OR compound flows:
                flow_list.append(flow_counter)
                flow_counter += 1
            else:
                flow_list.append(flow_counter)
        else: # If Unit changes
            if data['Unit'].iloc[i] in flow_dict: # If unit has already been added as a key to the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = flow_dict[data['Unit'].iloc[i]]
                flow_list.append(flow_counter)
            else: # If unit is NOT already associated with a value in the flow_dict
                flow_dict[data['Unit'].iloc[i-1]] = flow_counter
                flow_counter = 1
                flow_dict[data['Unit'].iloc[i]] = flow_counter
                flow_list.append(flow_counter)
                if data['Facies'].iloc[i]  == 'Compound lava (basic)': # Working around the situtation where the unit changes into a "compound lava" facies.
                    flow_counter += 1
                else:
                    continue

# Checking     
print(flow_list)
data['Flow#'] = flow_list
data.tail(25)

[1, 1, 1, '-', 2, 2, 2, '-', 3, 3, 3, '-', 4, 4, 4, 5, '-', 6, 7, 7, 7, 8, 9, 9, 9, '-', 10, 10, 10, '-', 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 18, 18, 18, '-', 19, 19, 19, '-', 20, 20, 20, 21, '-', 22, 22, 22, '-', 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 27, 27, 1, 1, 1, 2, 3, 3, 3, 4, 28, 28, 28, '-', 29, 30, 30, 30, 31, 32, 32, 32, '-', 5, '-', 5, 6, 6, 6, '-', 7, 8, 8, 8, 9, 10, 11, 11, 11, '-', 12, 12, 12, '-', 13, 13, 13, 14, 14, 14, '-', 15, 16, 16, 16, 17, 17, 17, 18, '-', 19, 19, 19, 20, 20, 20, 21, 21, 21, '-', 22, 22, 22, 23, 24, 24, 24, '-', 25, 25, 25, 26, 26, 26, '-', 27, 28, 28, 28, 29, 29, 29, '-', 30, 30, 30, 31, 32, 32, 32, '-', 33, 33, 33, 1, 2, 2, 2, 3, 4, 4, 4, 5]


,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion,Flow#
168,1-AV-001-PR,Pitanga,Simple lava (basic | L. crust),"1,44",NaN,NaN,28
169,1-AV-001-PR,Pitanga,Simple lava (basic | U. crust),"9,17","24,4","0,578278689",29
170,1-AV-001-PR,Pitanga,Simple lava (basic | massive interior),"14,11",NaN,NaN,29
171,1-AV-001-PR,Pitanga,Simple lava (basic | L. crust),"1,12",NaN,NaN,29
172,1-AV-001-PR,Pitanga,Siliciclastics,"4,24","4,24",NaN,-
173,1-AV-001-PR,Pitanga,Simple lava (basic | U. crust),"8,53","20,96","0,470896947",30
174,1-AV-001-PR,Pitanga,Simple lava (basic | massive interior),"9,87",NaN,NaN,30
175,1-AV-001-PR,Pitanga,Simple lava (basic | L. crust),"2,56",NaN,NaN,30
176,1-AV-001-PR,Pitanga,Compound lava (basic),"48,96","48,96",NaN,31
177,1-AV-001-PR,Pitanga,Simple lava (basic | rubbly flow top),"4,88","10,96","0,408759124",32


## Putting the algorithm inside a function for better functionality

## Others /// Notes: